In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select

In [2]:
#打开一个浏览器session
bro = webdriver.Chrome()
bro.get('http://www.ceic.ac.cn/history')

In [3]:
#选择start time,end time
DateS = input("请输入开始时间，格式yyyy-mm-dd")
bro.find_element_by_name("start").send_keys(DateS)
DateE = input("请输入结束时间")
bro.find_element_by_name("end").send_keys(DateE)
#填写纬度
bro.find_element_by_name('weidu1').send_keys("3°51′N")    
bro.find_element_by_name('weidu2').send_keys("53°33′N")   
#填写经度
bro.find_element_by_name('jingdu1').send_keys("73°33′E")
bro.find_element_by_name('jingdu2').send_keys("135°05′E")
#click search button
bro.find_element_by_id('search').click()

In [4]:
# 使用xpath选择器会出错，原因是“下一页”项目在无序列表ul中的顺序不固定

count = 0
earthquakeData = pd.DataFrame()
while True:
    count += 1
    print("正在抓取第{}个页面".format(count))
    time.sleep(3)
    earthquake = pd.read_html(bro.page_source, header=0)[0]
    earthquakeData = earthquakeData.append(earthquake)
    try:
        next_page = bro.find_elements_by_css_selector("li a[title='下一页']") 
        next_page[0].click()
    except IndexError:
        print("翻页结束")
        break

正在抓取第1个页面
正在抓取第2个页面
正在抓取第3个页面
正在抓取第4个页面
正在抓取第5个页面
翻页结束


In [5]:
earthquakeData

,震级(M),发震时刻(UTC+8),纬度(°),经度(°),深度(千米),参考位置
0,3.1,2021-07-09 10:41:58,45.77,89.66,7,新疆阿勒泰地区富蕴县
1,3.0,2021-07-09 05:04:20,39.53,76.94,17,新疆喀什地区伽师县
2,3.1,2021-07-08 21:51:14,40.54,82.88,18,新疆阿克苏地区沙雅县
3,4.9,2021-07-08 21:23:23,34.74,98.00,10,青海果洛州玛多县
4,4.6,2021-07-08 21:05:02,41.65,81.01,12,新疆阿克苏地区温宿县
...,...,...,...,...,...,...
18,3.0,2021-06-07 20:13:12,22.19,112.44,8,广东江门市开平市
19,3.4,2021-06-05 06:12:22,25.59,99.98,11,云南大理州漾濞县
0,3.0,2021-06-04 09:02:24,27.39,107.46,10,贵州遵义市余庆县
1,4.9,2021-06-03 13:55:17,34.75,97.90,10,青海果洛州玛多县


In [7]:
earthquakeData.to_csv('./earthquake.csv', index = False, encoding="utf_8_sig")